In [ ]:
%%capture
!pip install --quiet pyvi
!pip install --quiet evaluate
!pip install -U sentence-transformers

In [ ]:
!pip install --quiet underthesea

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer , TrainingArguments, Trainer , AutoModelForQuestionAnswering , AdamW , get_scheduler , DataCollatorWithPadding
from datasets import load_dataset , concatenate_datasets
#from accelerate import Accelerator
from pyvi import ViTokenizer, ViPosTagger
import evaluate
from torch.utils.data import DataLoader
from accelerate import Accelerator
#from transformers import DataCollatorWithPadding
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit , train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from datasets import Dataset,DatasetDict
import collections
import evaluate
from huggingface_hub import notebook_login
# from underthesea import sent_tokenize ,  text_normalize , word_tokenize
import sentence_transformers
from sentence_transformers import SentenceTransformer, util
import re
import torch.nn as nn
import pandas as pd
import seaborn as sns
from underthesea import word_tokenize
SEED = 1337

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
checkpoint = "vinai/phobert-base-v2"

### LOAD PREPROCESS DATA AND PRETRAINED MODEL

In [ ]:
# data = load_dataset("json", data_files="/kaggle/input/ranking-4-uit/FULL_27K_TRAINING_RANKING_DATA_RANKING_4")
# data = concatenate_datasets([data0['train'], data5['train'], data10['train'], data15['train'], data20])
data = load_dataset("json", data_files="/kaggle/input/uit-final-ranking-data-top6/FULL_27K_TRAINING_RANKING_DATA_RANKING_4 (3)")
data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def segment_context (samples) :
    return {"new_context" : [word_tokenize(sample, format="text") for sample in samples["new_context"] ] }

In [ ]:
data_segment = data.map (segment_context , batched = True)
data_segment

In [ ]:
data_segment = data_segment["train"]

In [ ]:
cls_tok = tokenizer.special_tokens_map['cls_token']
sep_tok = tokenizer.special_tokens_map['sep_token']
def fix_input (sample) :
    return {"input" : cls_tok + ' ' + sample['claim'] + ' ' + sep_tok +  ' ' +sample['new_context'] + ' ' + sep_tok  }

In [ ]:
data_fix_input = data_segment.map (fix_input )

In [ ]:
def tokenize (sample) :
    return tokenizer(sample ["input"] , truncation= True , max_length = 256)
data_tokenize = data_fix_input.map (tokenize , batched = True )

In [ ]:
data_tokenize

In [ ]:
# a = []
# for i in range (len ( data_)) :
#      a.append ( len ( data_[i]['input_ids'] )) 
# a = np.array (a)
# sns.histplot (a)
# print (max(a))

In [ ]:
train_data = data_tokenize.remove_columns (['context', 'claim', 'verdict', 'evidence', 'domain', '__index_level_0__', 'new_context','input'])
train_data

In [ ]:
split_dataset = train_data.train_test_split (train_size = 0.8)
split_dataset

## Classification model

In [ ]:
class CustomPhoBert(nn.Module):
    def __init__(self, n_classes):
        super(CustomPhoBert, self).__init__()
        self.model = AutoModel.from_pretrained(checkpoint , num_labels = 3)
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)
#         self.drop = nn.Dropout(p=0.2)
#         self.fc = nn.Linear(self.bert.config.hidden_size, n_classes)
#         nn.init.normal_(self.fc.weight, std=0.02)
#         nn.init.normal_(self.fc.bias, 0)
        self.att = nn.Sequential(
                            nn.Linear(768 , 384),
                            nn.Tanh(),
                            nn.Linear ( 384 , 1),
                            nn.Softmax(dim=1)
                            )
        self.drop = nn.Dropout (0.2)
        self.fc = nn.Linear (768 , 3)
        self.mini_linear = nn.Linear (3 , 1)
        self._init_weights(self.att)
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
            
    def forward(self, **batch):
        
        
        a = self.model (**batch )
    #     last_hidden_state , pooler = a.last_hidden_state , a.pooler_output # x , 100 , 768 # x , 758  # batch ,tok_len , hiddensize
        last_hidden_state = a.last_hidden_state
        weights = self.att (last_hidden_state)   # bacth ,tok_len , 1
        feature = torch.sum(weights * last_hidden_state, dim=1) # batch , tok_len, hidden_size --> batch , hiddensize )
        drop_feature = self.drop (feature)
        output = self.fc(drop_feature) #   batch , label

#         q1 = torch.mean ( output[start_indices [0]: end_indices[0]] , dim =0 ).view (1,3) # minisize , label # 2,3
#         q2 = torch.mean ( output[start_indices [1]: end_indices[1]] , dim =0 ).view (1,3)
#         q3 = torch.mean ( output[start_indices[2]: end_indices[2]] , dim =0 ).view (1,3)
#         q4 = torch.mean ( output[start_indices[3]: end_indices[3]] , dim =0 ).view (1,3) 
#         logits = torch.concat ((q1,q2,q3,q4) ,0)
        
#        return logits
        return output
model = CustomPhoBert (3).to(device)

In [ ]:
epochs = 10
# device = torch.device ("cuda") if torch.cuda.is_available() else torch.device ("cpu" )
# model.to(device)
#Dataloader
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
train_dataloader = DataLoader (split_dataset["train"], batch_size= 32, shuffle=True , collate_fn = data_collator  )
val_dataloader = DataLoader (split_dataset["test"], batch_size= 64, shuffle=True , collate_fn = data_collator )
# test_dataloader = DataLoader (split_dataset["test"] , batch_size=16, shuffle=True , collate_fn = data_collator )
step_to_train = epochs * len (train_dataloader)
optimizer = torch.optim.AdamW (model.parameters () , lr = 3e-5)
scheduler = get_scheduler (name = "cosine" , 
                           optimizer = optimizer ,
                           num_warmup_steps=0,
                           num_training_steps = step_to_train
                          )
#accelerator
accelerator = Accelerator ()
train_dataloader ,val_dataloader ,model , optimizer= accelerator.prepare (train_dataloader  , val_dataloader , model , optimizer)

#optimzier and scheduler
#Metric
metric_train_acc = evaluate.load("accuracy")
metric_valid_acc = evaluate.load("accuracy")
metric_train_f1 = evaluate.load("f1")
metric_valid_f1 = evaluate.load("f1")

### LOAD FINETUNE MODEL IF YOU HAVE ONE

In [ ]:
# model.load_state_dict(torch.load("/kaggle/input/viedeberta-small-model-ranking-10000/viedeberta_small_model_ranking"))

In [ ]:
# import gc
# torch.cuda.empty_cache()
# gc.collect()

In [ ]:
from tqdm.auto import tqdm
best_val_acc = 0
i=0
for epoch in range (epochs) :
    model.train ()
    losses = []
    for batch in tqdm ( train_dataloader )  :
        labels = batch.pop ("labels")
        
        logits = model(**batch)

        criterion = nn.CrossEntropyLoss()
        #print (outputs.shape , labels.shape)
        loss = criterion(logits,labels)

        accelerator.backward (loss)

        optimizer.step ()
        scheduler.step ()
        optimizer.zero_grad ()

        losses.append (loss.item())
        predict = torch.argmax (logits, dim = -1)
        #print ( predict.shape)
        metric_train_acc.add_batch (references=labels, predictions = predict)
        metric_train_f1.add_batch (references=labels, predictions = predict)
    print ("Epoch : {} , Loss : {} , Accuracy : {} , F1 : {}".format (epoch + 1 , np.mean( losses) , metric_train_acc.compute ()['accuracy'] , metric_train_f1.compute(average="weighted")['f1'] ) )
    
    model.eval()
    val_losses = []
    for batch in tqdm ( val_dataloader ) :
        with torch.no_grad () :
            labels = batch.pop ("labels")
            logits = model.forward (**batch)
            
            criterion = nn.CrossEntropyLoss()
            #print (outputs.shape , labels.shape)
            loss = criterion(logits,labels)
            val_losses.append (loss.item())
        
            predict = torch.argmax (logits, dim = -1)
            metric_valid_acc.add_batch (references=labels, predictions = predict)
            metric_valid_f1.add_batch (references=labels, predictions = predict )
            
    val_acc = metric_valid_acc.compute ()['accuracy']
    if val_acc > best_val_acc :
        torch.save(model.state_dict(), "Phobert_v2_best_model_full_data_{}".format (i))
        best_val_acc = val_acc
        i+=1
    print ("Epoch : {}, Val Loss: {} , Validation  ACC Result : {} , Validation F1 Result :{}".format (epoch + 1, np.mean( val_losses) , val_acc , metric_valid_f1.compute (average="weighted")['f1'] ))

In [ ]:
data_pd = data.to_json("data")